# Penetration testing notebook

Use this notebook to test handcrafted queries directly against the sakila demo database without going to the LLM. If you're here, please take a moment to view our [Security Policy](https://github.com/amoffat/HeimdaLLM/security/policy) when it comes to disclosing vulnerabilities.


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import logging

import structlog

logging.basicConfig(level=logging.DEBUG)
structlog.configure(logger_factory=structlog.stdlib.LoggerFactory())

import sqlite3
import sys
import time
from datetime import date
from pathlib import Path

import pandas as pd

from heimdallm.bifrosts.sql.sqlite.select.bifrost import Bifrost

THIS_DIR = Path().resolve()
sys.path.insert(0, THIS_DIR)

You can browse the Sakila Demo Database [here](./sakila.sqlite3) (Note: the sqlite
VSCode extension doesn't seem to work in Github Codespaces)


In [4]:
conn = sqlite3.connect("sakila.sqlite3")
db_schema = open("sakila-schema.sql", "r").read()

In [5]:
from constraints.customer import DataConstraints

validator = DataConstraints()
bifrost = Bifrost.validation_only(DataConstraints())

In [6]:
def query(untrusted_input):
    trusted_sql_query = bifrost.traverse(untrusted_input, autofix=True)

    # according to the Sakila database, this is roughly the latest meaningful date that
    # could be considered "now", so we'll hardcode that, so that the database is always
    # relevant. our prompt envelope has instructed the LLM to reference the `:timestamp`
    # placeholder in the query if it needs the current time.
    now = time.mktime(date.fromisoformat("2005-09-05").timetuple())
    params = {"timestamp": now, "customer_id": 148}

    df = pd.read_sql_query(trusted_sql_query, conn, params=params)
    display(df)

Craft your query here to execute and see the results.


In [9]:
untrusted_input = "SELECT id from film;"
query(untrusted_input)

INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Traversing untrusted input     autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Wrapping input in prompt envelope autofix=True
DEBUG:heimdallm.bifrost:2023-09-11 22:39:39 [debug    ] Produced prompt envelope       autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Sending envelope to LLM        autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Received raw result from LLM   autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Unwrapping prompt envelope     autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Unwrap succeeded               autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Parsing result via grammar     autofix=True


INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Parse succeeded                autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Trying constraint validator    autofix=True validator=DataConstraints
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Autofixing parse tree and reconstructing the input autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Re-parsing reconstructed output autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Reconstruction succeeded       autofix=True
INFO:heimdallm.bifrost:2023-09-11 22:39:39 [info     ] Validating parse tree          autofix=True
ERROR:heimdallm.bifrost:2023-09-11 22:39:39 [error    ] Validation failed              autofix=True
Traceback (most recent call last):
  File "/home/amoffat/work/heimdallm/heimdallm/bifrost.py", line 132, in traverse
    raise validation_exc
  File "/home/amoffat/work/heimdallm/heimdallm/bifrost.py", line 113, in traverse
    trusted_llm_output, tree = self

MissingRequiredIdentity: Missing one required identities: {rental.customer_id=:customer_id, payment.customer_id=:customer_id, customer.customer_id=:customer_id}